IMPORT LIBRARIES

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

READ IN CSV FILES

In [2]:
links_df = pd.read_csv('links.csv')
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
tags_df = pd.read_csv('tags.csv')

GENERAL **INFO** ON EVERY DATAFRAME

In [3]:
print('\n LINKS DataFrame Info:')
links_df.info() 

print('\n MOVIES DataFrame Info:')
movies_df.info()

print('\n RATINGS Dataframe Info:')
ratings_df.info()

print('\n TAGS Dataframe Info:')
tags_df.info()


 LINKS DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB

 MOVIES DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB

 RATINGS Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 

LOOK AT **RANDOM SAMPLES**  

In [11]:
def df_randomsamples(sample_size = 5, random_state = None, **dfs):
    '''
    Get side-by-side random samples from multiple DataFrames.
    
    Parameters:
        sample_size (int): Number of rows to sample from each DataFrame. Default = 5.
        random_state (int): Seed for random sampling. Default = None. 
        **dfs (pd.DataFrame): Keyword arguments where the key is the DataFrame name and the value is the DataFrame itself.
    Returns:
        pd.Dataframe: A DataFrame containing side-by-side random samples from each input DataFrame.
    '''
    random_samples = [(df_name, df.sample(n=sample_size, random_state=random_state)) for df_name, df in dfs.items()]
    
    # Display sampled DataFrames side by side
    for df_name, sampled_df in random_samples:
        print(f"DataFrame: {df_name}")
        display(sampled_df)


In [16]:
df_randomsamples(sample_size = 5, random_state = 42, tags_df = tags_df, movies_df = movies_df, ratings_df = ratings_df)

DataFrame: tags_df


,userId,movieId,tag,timestamp
2023,474,5644,baseball,1138039381
2587,477,4226,twist ending,1256051079
3222,567,112852,unlikely hero,1525285378
1263,474,1028,Disney,1137375815
781,424,1258,atmospheric,1457843344


DataFrame: movies_df


,movieId,title,genres
6213,45635,"Notorious Bettie Page, The (2005)",Drama
1056,1373,Star Trek V: The Final Frontier (1989),Action|Sci-Fi
4891,7325,Starsky & Hutch (2004),Action|Comedy|Crime|Thriller
346,389,"Colonel Chabert, Le (1994)",Drama|Romance|War
5353,8920,"Country Girl, The (1954)",Drama


DataFrame: ratings_df


,userId,movieId,rating,timestamp
67037,432,77866,4.5,1335139641
42175,288,474,3.0,978465565
93850,599,4351,3.0,1498524542
6187,42,2987,4.0,996262677
12229,75,1610,4.0,1158989841


In [15]:
tags_df.sample(n = 5, random_state = 42)

,userId,movieId,tag,timestamp
2023,474,5644,baseball,1138039381
2587,477,4226,twist ending,1256051079
3222,567,112852,unlikely hero,1525285378
1263,474,1028,Disney,1137375815
781,424,1258,atmospheric,1457843344


In [ ]:
ratings_df.sample(n = 20)

In [ ]:
movies_df.sample(n = 20)